# Adding Brick tags for legacy BMS tags
Following script is for finding Brick Ontology Classifications for a Johnson Control BMS.

In [ ]:
# !pip install googletrans -y
# !pip install googletrans==3.1.0a0 openpyxl

In [4]:
import pandas as pd
import json
# from googletrans import Translator
import numpy as np

In [3]:
# dest_lang = 'en'

# translator = Translator()
# translation = translator.translate('Welkom bij slimme gebouwen!', dest=dest_lang)
# print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

## Selecting data points which match the naming convention

The pattern of the naming convention should be understood by this example --> (XX) 202.CV-03D-- 

Naming convention is divided into four parts as,
1. Building Number (XX),
2. System Number (202), 
3. Code (CV), and
4. Last_Letter (D). 

Importing the BMS Object List (extracted from BMS) that comply with the naming convention. At this point we only consider the data points which follow the naming convention, and therefore this imported file is already a processed file by applying a Regular Expression. Pandas or OpenRefine [https://openrefine.org/] can be used to do this pre-processing. Note that the actual Building Number is replaced to adhere with data sharing guideline.

In [ ]:
df = pd.read_excel('./inputs/split_ids_with_data_en.xlsx')
df = df.drop(['Unnamed: 0', 'Suffix'], axis=1)
df = df.rename(columns={'Number': 'Num'})
df['Num'] = df["Num"].map(str)
df['Num'] = df['Num'].str.zfill(3)

df = df.astype({'Last_Letter':'string'})
df['Last_Letter']=df['Last_Letter'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()

df.head()

Now we have three mapping tables, each describing,
1. System Number, 
2. Code, and 
3. Last_Letter.

# Applying System Number Code

In [7]:
df_nums = pd.read_excel('./inputs/codes/numbers_processed.xlsx')
df_nums = df_nums.astype({'Num':'string'})
df_nums['Num'] = df_nums['Num'].str.zfill(3)
df_nums.head()

,Num,Num Description (NL),Num Description (EN)
0,001,Ketel 1,Boiler 1
1,002,Ketel 2,Boiler 2
2,011,Transportsysteem ketel 1,Transport system boiler 1
3,012,Transportsysteem ketel 2,Transport system boiler 2
4,021,Warm water groep 1,Hot water group 1


#### Applying the numbers to all numbers from 001-999. 
Full list is generated from code under '## Tag2: Add tag names based on number_types.txt file'

In [8]:
df_num_cls = pd.read_json('./inputs/numbers_en_full_Brick.json',  typ='series')
df_num_cls = df_num_cls.to_frame().reset_index()
df_num_cls.columns = ['Num', 'Num Class Brick']
df_num_cls = df_num_cls.astype({'Num':'string'})
df_num_cls['Num'] = df_num_cls['Num'].str.zfill(3)
df_num_cls

,Num,Num Class Brick
0,001,Boiler
1,002,Boiler
2,003,Boiler
3,004,Boiler
4,005,Boiler
...,...,...
883,900,Water_Loss_Alarm
884,901,Water_Loss_Alarm
885,904,Compressor
886,905,Server_Room


In [9]:
df_full_nums = pd.merge(df_num_cls, df_nums, how='left', on='Num')
# df_full_nums.to_excel('./outputs/all_nums.xlsx')
# df_full_nums.head()

### Using the original df and add matched nums to it

In [10]:
map_num = pd.merge(df, df_full_nums, how='left', on='Num')
map_num.head()
print(f'Length:{len(map_num)}')

Length:763


# Applying the middle letter code

In [11]:
df_code = pd.read_excel('./inputs/codes/codes_processed.xlsx')

df_code['Code'] = df_code['Code'].map(lambda x: x.strip('- '))
# df_code['Code'].str.strip()
df_code.head()

,Code,Code Description (NL),Code Description (EN),Code Brick Class
0,AKM,Absorptiekoelmachine,Absorption chiller,Absorption_Chiller
1,BA,Brandmelding,Fire alarm,Fire_Alarm
2,BG,Breekglaasje,Break glass,Fire_Alarm_Manual_Call_Point
3,BK,NaN,Firedamper,Damper
4,BKA,Brandklep afvoer,Fire damper exhaust,Exhaust_Damper


### Using the df from above step (matched nums) and add matched codes to it

In [12]:
map_num['Code'] = map_num['Code'].str.lower()
df_code['Code'] = df_code['Code'].str.lower()

map_num_code = pd.merge(map_num, df_code, how='left', on='Code')
map_num_code.head()
print(f'Length:{len(map_num_code)}')

Length:763


# Applying the last letter code

In [13]:
df_last_letter = pd.read_excel('./inputs/codes/last_letter_processed.xlsx')
df_last_letter['Last_Letter'] = df_last_letter['Last_Letter'].map(lambda x: x.strip('- '))
df_last_letter['Last_Letter'].str.strip()
df_last_letter.head()

,Last_Letter,Last_Letter Description (NL),Last_Letter Description (EN),Last_Letter Brick Class
0,A,Alarm,Alarm,Alarm
1,AB,brandmelding RK-01 BMC,Fire alarm RK-01 BMC,Fire_Sensor
2,AI,installatie automaten,Fuse,Switchgear
3,AN,netwachter,Power phase monitor (alarm),Power_Loss_Alarm
4,AS,sabotage,sabotage,NaN


### Using the df from above step (matched nums and codes) and add matched last letters to it.
This is going to be the final output

In [14]:
df_last_letter['Last_Letter'] = df_last_letter['Last_Letter'].str.lower()
map_num_code['Last_Letter'] = map_num_code['Last_Letter'].str.lower()

map_num_code_lletter = pd.merge(map_num_code, df_last_letter, how='left', on='Last_Letter')
map_num_code_lletter.head()
print(f'Length:{len(map_num_code_lletter)}')

Length:766


In [15]:
map_num_code_lletter.columns

Index(['Item Reference', 'Object ID', 'Object Type', 'Name', 'Num', 'Code',
       'Last_Letter', 'Description (NL)', 'Description (EN)',
       'Num Class Brick', 'Num Description (NL)', 'Num Description (EN)',
       'Code Description (NL)', 'Code Description (EN)', 'Code Brick Class',
       'Last_Letter Description (NL)', 'Last_Letter Description (EN)',
       'Last_Letter Brick  Class'],
      dtype='object')

In [328]:
# map_num_code_lletter = map_num_code_lletter[['Item Reference', 'Object ID', 'Object Type', 'Name', 'Num', 'Code',
#        'Description (EN)',
#        'Num Class Brick', 'Num Description (EN)',
#        'Num Brick', 'Code Description (EN)',
#        'Code Brick Class', 'Last_Letter Description (EN)',
#        'Last_Letter Brick  Class']]

In [ ]:
map_num_code_lletter.head()

In [37]:
map_num_code_lletter.to_excel('./outputs/brick-mappings.xlsx')

In [331]:
map_num_code_lletter.columns
map_num_code_lletter = map_num_code_lletter[['Item Reference', 'Object ID', 'Object Type', 'Name', 'Num', 'Code',
       'Last_Letter', 'Num Class Brick','Code Brick Class','Last_Letter Brick  Class']]

Group by Equipment

In [332]:
# grouped = map_num_code_lletter.groupby('Num Class Brick')

# for name, group in grouped:
#     filename = f"./outputs/groups/{name}.xlsx"
#     group.to_excel(filename, index=False)
